In [1]:
import numpy as np
from scipy.stats import norm
from math import pi
%run Europea_Options.ipynb

In [2]:
class Exotics(Black_Scholes_Merton):
    
    def __init__(self,S_t,K,T,sigma,r,D,n):
        
        Black_Scholes_Merton.__init__(self,S_t,K,T,sigma,r,D)
        self.n = n 
        
        self.d_1 = ((np.log(self.S_t / (np.power(self.K,1/self.n)))) + ((self.r - self.D) + ((self.n - 1/2) * np.power(self.sigma,2)) * self.T )) / (self.sigma * np.sqrt(self.T))  
        self.d_2 = self.d_1 - self.n * self.sigma * np.sqrt(self.T)
        
    
    def PowerOptionCall(self):
        
        return (np.power(self.S_t, self.n) * np.exp(- self.n * self.D * self.T) * norm.cdf(self.d_1)) - self.K * np.exp(- self.n * (self.r + ((1/2) * (self.n - 1) * np.power(self.sigma,2))) * self.T) * norm.cdf(self.d_2)
    
    
    def PowerOptionPut(self):
        
        call = self.PowerOptionCall()
        
        
        return call - self.S_t * np.exp(- self.D * self.T) + self.K * np.exp(-self.r * self.T)
    
    
    
    def Asset_Or_Nothing_Call(self):
        
        return self.S_t * np.exp(-self.D * self.T) * norm.cdf(self.d1)
    
    
    
    def Asset_Or_Nothing_Put(self):
        
        return self.S_t * np.exp(-self.D * self.T) * norm.cdf(-self.d1)
    
    
    
    def Digital_Call(self):
        
        return np.exp(-self.r * self.T) * norm.cdf(self.d2)
    
    
    def Digital_Put(self):
        
        return np.exp(-self.r * self.T) * norm.cdf(-self.d2) 
    
    
    
    def shoot_display(self):
        
        import pandas as pd
        
        return pd.DataFrame(data={'Valeur_PowerOptionCall':[self.PowerOptionCall()],
                                 'Valeur_PowerOptionPut':[self.PowerOptionPut()],
                                 'Valeur_Asset_Or_Nothing_Call':[self.Asset_Or_Nothing_Call()],
                                 'Valeur_Asset_Or_Nothing_Put':[self.Asset_Or_Nothing_Put()],
                                 'Valeur_Digital_Call':[self.Digital_Call()],
                                 'Valeur_Digital_Put':[self.Digital_Put()]}).T.rename(columns={0:'Valeurs'})

In [3]:
option = Exotics(100,75,1,0.3,0.2,0.1,2)

In [4]:
option.shoot_display()

,Valeurs
Valeur_PowerOptionCall,8141.360551
Valeur_PowerOptionPut,8112.281616
Valeur_Asset_Or_Nothing_Call,83.732514
Valeur_Asset_Or_Nothing_Put,6.751228
Valeur_Digital_Call,0.715022
Valeur_Digital_Put,0.103709
